<a href="https://colab.research.google.com/github/shake/colab-Llama-2-ipynb/blob/main/Fine_Tune_Llama2%2B%E8%AF%B4%E6%98%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Fine-Tune Llama 2 Model**

**There are two main Fine-Tunining techniques**
1. Supervised Fine-Tuning (SFT)
2. Reinforcement Learning from Human Feedback (RLHF)

Supervised Fine-Tuning (SFT):

In Supervised Fine-Tuning the model is trained/fine-tuned on a dataset of instruction and responses. It adjusts the weights in the LLM to minimize the difference between the generated answers and ground truth responses, acting as labels

**In this notebook, Supervised Fine-Tuning is performed**

**Step:01 Install All the Required Libraries**

In [1]:
%%capture
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate==0.21.0 peft==0.4.0   trl==0.4.7 datasets==2.13.0 safetensors>=0.3.1
!pip install -q -U huggingface_hub sentencepiece einops nvidia-ml-py3 tiktoken bitsandbytes

**Step 02: Set the enviornment as Hugging Face Token**

In [2]:
# import 密钥，token
%%capture
from google.colab import userdata
hf_token = userdata.get('huggingface')
!git config --global credential.helper store
!huggingface-cli login --token $hf_token --add-to-git-credential

**Step 03: Import All the Required Libraries**

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

**Step 04: Fine-Tune the Llama 2 model using Supervised Fine-Tuning (SFT)**

There are three ways in which we can fine-tune the model using Supervised Fine-Tuning (SFT).

1. Full Fine-Tuning

2. LoRA

3. QLoRA

Full Fine-Tuning: With Full Fine-Tuning we are going to use the entire model, We will train all the weights in the model which is very costly.

LoRA: In LoRA instead of training all the weights, we will add some adapters in some layers and we will only train the added weights, which will reduce the cost of training the model because we are only training like 1% 2% of the entire weights


QLoRA: QLoRA which uses LoRA but here we use a model which has been quantized. If the LLM model is occupying 16bits on the disk, in QLoRA they will be quantized into 4bits so a lots of precision will be lose.

**In this notebook, I am fine-tuning the Llama 2 model with 7 billion parameters, T4-GPU has 15GB of VRAM (GPU Memory) which is barely enough to store the Llama2-7b's weights (7b x 2bytes = 14GB in FP16). Plus we also need to consider the overhead due to optimizer states, gradients and forward activatons.**

**To reduce the VRAM usage (GPU Memory Usage) we will fine-tune the Llama 2 model in 4bit precision which is why we will use QLoRA here**

In [13]:
 # Model
base_model = "chenshake/Llama-2-7b-chat-hf"
#Fine-tune model name
new_model = "Lama-2-7b-chat-hf-Qlora"
#Load the Dataset from hugging face
dataset = load_dataset("MMoin/mini-platypus", split="train")
#Tokenizer
#Load the tokenizer from Llama 2
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
#In Llama2 we dont have the padding token which is a very big problem, because we have a dataset with different number of tokens in each row.
#So, we need to pad it so they all have the same length and here i am using end of sentence token and this will have an impact on the generation of our model
#I am using End of Sentence token for fine-tuning
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

In [5]:
dataset

Dataset({
    features: ['instruction', 'output'],
    num_rows: 1000
})

In [ ]:
dataset.to_pandas()

In [ ]:
#Configration of QLoRA
#Quantization Configuration
#To reduce the VRAM usage we will load the model in 4 bit precision and we will do quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #Quant type
    #We will use the "nf4" format this was introduced in the QLoRA paper
    bnb_4bit_quant_type="nf4",
    #As the model weights are stored using 4 bits and when we want to compute its only going to use 16 bits so we have more accuracy
    bnb_4bit_compute_dtype=torch.float16,
    #Quantization parameters are quantized
    bnb_4bit_use_double_quant=True,
)


# LoRA configuration
peft_config = LoraConfig(
    #Alpha is the strength of the adapters. In LoRA, instead of training all the weights, we will add some adapters in some layers and we will only
    #train the added weights
    #We can merge these adapters in some layers in a very weak way using very low value of alpha (using very little weight) or using a high value of alpha
    #(using a big weight)
    #15 is very big weight, usually 32 is considered as the standard value for this parameter
    lora_alpha=15,
    #10% dropout
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Cast the layernorm in fp32, make output embedding layer require grads, add the upcasting of the lmhead to fp32
#prepare_model_for_kbit_training---> This function basically helps to built the best model possible
model = prepare_model_for_kbit_training(model)

In [ ]:
# Set training arguments
training_arguments = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,#3,5 good for the Llama 2 Model
        per_device_train_batch_size=4,# Number of batches that we are going to take for every step
        gradient_accumulation_steps=1,
        evaluation_strategy="steps",#Not helpful because we donot want to evaluate the model we just want to train it
        eval_steps=1000,
        logging_steps=25,
        optim="paged_adamw_8bit",#Adam Optimizer we will be using but a version that is paged and in 8 bits, so it will lose less memory
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        warmup_steps=10,
        report_to="tensorboard",
        max_steps=-1, # if maximum steps=2, it will stop after two steps
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=dataset,#No separate evaluation dataset, i am using the same dataset
    peft_config=peft_config,
    dataset_text_field="instruction",
    max_seq_length=512,# In dataset creation we put a threshold 2k for context length (input token limit) but we dont have enough VRAM unfortunately it will take a lot of VRAM to put everything into memory so we are just gonna stop at 512
    tokenizer=tokenizer,
    args=training_arguments,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [ ]:
# Run text generation pipeline with our model
#Input Prompt
prompt = "What is a large language model?"
#Wrap the prompt using the right chat template
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
#Using Pipeline from the hugging face
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
#Trim the response, remove instruction manually
print(result[0]['generated_text'][len(instruction):])

**The response keeps repeating because of the padding technique, end of sentence token, If you dont want to see this behaviour please use a different padding technique**

In [ ]:
# Empty VRAM
del model
del pipe
#del trainer
import gc
gc.collect()
gc.collect()

**Merge the Base Model with the Trained Adapter**

---



In [14]:
# Reload model in FP16 and merge it with LoRA weights
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
#Reload the Base Model and load the QLoRA adapters
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 63.06 MiB is free. Process 32100 has 14.68 GiB memory in use. Of the allocated memory 14.44 GiB is allocated by PyTorch, and 120.06 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

**Push the Fine-Tuned Model and Tokenizer to the Hugging Face Hub**

In [13]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [13]:
import huggingface_hub
#repo_id = f'{huggingface_hub.whoami()["name"]}/{peft_name}'
repo_id = f'{huggingface_hub.whoami()["name"]}/{new_model}'

In [14]:
print(repo_id)

chenshake/Lama-2-7b-chat-hf-QLoRA


In [ ]:
trainer.model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)